In [3]:
#imports for getting and sorting files beased on date modified.
import os
from os.path import isfile, join
from PIL import Image
from PIL.ExifTags import TAGS
import win32com.client



mypath = os.getcwd()
onlyfiles = [f for f in os.listdir(mypath) if isfile(join(mypath, f))]
onlyfiles.remove("Video Processer.ipynb")
#print(onlyfiles)

#This is where the meta data for the files are stored and processed. 
"""
NOTE: If you do not wish to sort based on 'Date Modified' 
    - Look into the coloumns list and pic the position of the element you would like to use to sort by.
"""
sh=win32com.client.gencache.EnsureDispatch('Shell.Application',0)
ns = sh.NameSpace(mypath)
colnum = 0
columns = []
fileDates = []
while True:
    colname=ns.GetDetailsOf(None, colnum)
    if not colname:
        break
    columns.append(colname)
    colnum += 1

for item in ns.Items():
    if (ns.GetDetailsOf(item,0)+".jpg") in onlyfiles:
        fileDates.append([ns.GetDetailsOf(item,0)+".jpg",ns.GetDetailsOf(item,3)])
    elif (ns.GetDetailsOf(item,0)+".mp4") in onlyfiles:
        fileDates.append([ns.GetDetailsOf(item,0)+".mp4",ns.GetDetailsOf(item,3)])

fileDates.sort(key=lambda x: x[1])

#print(fileDates)




In [4]:
import cv2
import numpy as np

# function called by trackbar, sets the next frame to be read
def getFrame(frame_nr):
    global video
    video.set(cv2.CAP_PROP_POS_FRAMES, frame_nr)

    # Get the next videoframe
    ret, frame = video.read()

    # show frame
    if ret:
        cv2.imshow("Video", frame)
    


def getImageFromVid(vidName, counterName):

    # open video
    global video 
    video = cv2.VideoCapture(vidName)
    # get total number of frames
    nr_of_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    # create display window
    cv2.namedWindow("Video")
    # set wait for each frame, determines playbackspeed
    playSpeed = 50
    # add trackbar
    cv2.createTrackbar("Frame", "Video", 0,nr_of_frames,getFrame)

    first = True
    # main loop, runs untill exited or frame saved
    while 1:
        #Display the first frame then wait for trackbar to be updated
        if first:
            ret, frame = video.read()

            # show frame, break the loop if no frame is found
            if ret:
                cv2.imshow("Video", frame)
                # update slider position on trackbar
                # NOTE: this is an expensive operation, remove to greatly increase max playback speed
                cv2.setTrackbarPos("Frame","Video", int(video.get(cv2.CAP_PROP_POS_FRAMES)))
            else:
                break

            first = False

        # display frame for 'playSpeed' ms, detect key input
        key = cv2.waitKey(playSpeed)

        # stop playback when q is pressed
        if key == ord('q'):
            break
        # save frame if s is pressed
        if key == ord('s'):
            ret, frame = video.read()
            cv2.imwrite(str(counterName)+".jpg",frame)
            break



    # release resources
    video.release()
    cv2.destroyAllWindows()

In [5]:
#This is where the sorted images and videos are renamed from '1.jpg/mp4'...
fileNameCounter = 1
for i in fileDates:
    name = i[0].split(".")
    if name[1] == "jpg":
        os.rename(mypath+"\\" + (i[0]),mypath+"\\" +(str(fileNameCounter)+".jpg"))
    else:
        getImageFromVid(i[0],fileNameCounter)
    fileNameCounter += 1


